# NFL macro trends report (1999–present)

This notebook builds a league-wide view of offensive trends using your on-disk Parquet datasets:
- Play-by-play: `data/silver/pbp`
- Schedules (optional): `data/silver/schedules`

It computes and visualizes:
- Pace: plays per game, estimated seconds per play
- Pass tendency and pass rate over expected (PROE)
- Efficiency: EPA/play and success rate (overall, pass, rush)
- Formations/tempo: shotgun rate, no-huddle rate
- Passing depth: air yards
- Environment effects: dome vs outdoors, and grass vs turf (points/game, EPA/play, explosive play rate)

Outputs are saved to `reports/macro_trends/` for reuse in slides and docs.


### Environment setup
Install and import analysis libraries used throughout the notebook. If your environment already has them, the install cell will no-op.


In [1]:
pip install -q pandas polars pyarrow duckdb matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


### Optional: limit years for remote fallback
If local parquet is missing, we’ll stream remote PBP files. You can limit years to speed things up by setting `NFL_YEARS` (e.g., `2015-2024`).


In [ ]:
# Set a narrower range to speed up first run if desired
# os.environ["NFL_YEARS"] = "2015-2024"


### Optional: materialize remote PBP to local Parquet
To avoid re-downloading on every run, write the currently-selected years from `pbp` view to `data/silver/pbp/year=YYYY/`. Safe to skip if you prefer streaming.


### Data connections
We’ll register Parquet datasets as DuckDB views (`pbp`, `schedules`) for SQL-style queries without loading everything into memory. This supports incremental analysis.


In [5]:
# Imports and config
import os
from pathlib import Path

import duckdb as ddb
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

DATA_ROOT = Path("data/silver")
REPORT_DIR = Path("reports/macro_trends")
REPORT_DIR.mkdir(parents=True, exist_ok=True)

con = ddb.connect()

# Register Parquet datasets as views (with remote fallback if local parquet is missing)
local_files = [p.as_posix() for p in (DATA_ROOT / "pbp").glob("**/*.parquet")]
if local_files:
    con.execute(f"""
        CREATE OR REPLACE VIEW pbp AS
        SELECT * FROM read_parquet('{(DATA_ROOT / "pbp" / "**/*.parquet").as_posix()}');
    """)
else:
    years_env = os.getenv("NFL_YEARS", "1999-2024")

    def _parse_years_arg(years: str):
        years = years.strip()
        if "-" in years:
            start, end = years.split("-")
            return list(range(int(start), int(end) + 1))
        return [int(x) for x in years.split(",") if x.strip()]

    YEARS = _parse_years_arg(years_env)
    urls = [
        f"https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{y}.parquet"
        for y in YEARS
    ]
    files_sql = ", ".join(repr(u) for u in urls)
    con.execute("INSTALL httpfs; LOAD httpfs;")
    con.execute(f"""
        CREATE OR REPLACE VIEW pbp AS
        SELECT * FROM read_parquet({files_sql});
    """)
    print(f"Loaded remote PBP for years: {YEARS}")

# Schedules is optional; some fields like roof/surface may come from pbp already
if (DATA_ROOT / "schedules").exists():
    con.execute(f"""
        CREATE OR REPLACE VIEW schedules AS
        SELECT * FROM read_parquet('{(DATA_ROOT / "schedules" / "**/*.parquet").as_posix()}');
    """)
else:
    con.execute("CREATE OR REPLACE VIEW schedules AS SELECT NULL WHERE FALSE;")

print("Views ready: pbp, schedules")

BinderException: Binder Error: No function matches the given name and argument types 'read_parquet(VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR)'. You might need to add explicit type casts.
	Candidate functions:
	read_parquet(VARCHAR, can_have_nan : BOOLEAN, filename : ANY, union_by_name : BOOLEAN, debug_use_openssl : BOOLEAN, hive_partitioning : BOOLEAN, parquet_version : VARCHAR, encryption_config : ANY, hive_types_autocast : BOOLEAN, binary_as_string : BOOLEAN, explicit_cardinality : UBIGINT, compression : VARCHAR, file_row_number : BOOLEAN, hive_types : ANY, schema : ANY)
	read_parquet(VARCHAR[], can_have_nan : BOOLEAN, filename : ANY, union_by_name : BOOLEAN, debug_use_openssl : BOOLEAN, hive_partitioning : BOOLEAN, parquet_version : VARCHAR, encryption_config : ANY, hive_types_autocast : BOOLEAN

### Why we’re setting up these tables
We’ll materialize season-level tables so downstream plotting is fast and reproducible. This keeps notebook runs quick even on large PBP.


### Why we’re setting up these tables
We’ll materialize season-level tables so downstream plotting is fast and reproducible. This keeps notebook runs quick even on large PBP.


In [ ]:
-- Season aggregates for macro trends
CREATE OR REPLACE TABLE season_trends AS
WITH base AS (
  SELECT
    season::INT AS season,
    game_id,
    -- pace proxies
    COUNT(*) FILTER (WHERE play IN (1)) OVER (PARTITION BY season, game_id) AS plays_in_game,
    -- pass/rush flags
    pass::INT AS is_pass,
    rush::INT AS is_rush,
    -- tempo / formation
    shotgun::INT AS is_shotgun,
    no_huddle::INT AS is_no_huddle,
    -- depth and efficiency
    air_yards,
    epa,
    success::INT AS is_success,
    -- environment
    COALESCE(roof, game_stadium, stadium, '') AS roof_like,
    surface
  FROM pbp
  WHERE season >= 1999 AND season_type = 'REG' AND play_deleted IS NULL
), by_game AS (
  SELECT
    season,
    game_id,
    SUM(1) AS plays,
    SUM(is_pass) AS pass_plays,
    SUM(is_rush) AS rush_plays,
    AVG(is_shotgun) AS shotgun_rate,
    AVG(is_no_huddle) AS no_huddle_rate,
    AVG(NULLIF(air_yards, NULL)) AS mean_air_yards,
    AVG(epa) AS epa_play,
    AVG(CASE WHEN is_pass=1 THEN epa END) AS epa_pass,
    AVG(CASE WHEN is_rush=1 THEN epa END) AS epa_rush,
    AVG(is_success) AS success_rate,
    AVG(CASE WHEN is_pass=1 THEN is_success END) AS pass_success,
    AVG(CASE WHEN is_rush=1 THEN is_success END) AS rush_success,
    AVG(CASE WHEN roof_like ILIKE '%dome%' OR roof_like ILIKE '%closed%' THEN 1 ELSE 0 END) AS is_dome,
    CASE WHEN surface ILIKE '%turf%' OR surface ILIKE '%artificial%' THEN 'turf' ELSE 'grass' END AS surface_class
  FROM base
  GROUP BY season, game_id, surface_class
)
SELECT
  season,
  COUNT(DISTINCT game_id) AS games,
  AVG(plays) AS avg_plays_per_game,
  AVG(pass_plays::DOUBLE)/NULLIF(AVG(plays::DOUBLE),0) AS pass_rate,
  AVG(shotgun_rate) AS shotgun_rate,
  AVG(no_huddle_rate) AS no_huddle_rate,
  AVG(mean_air_yards) AS air_yards,
  AVG(epa_play) AS epa_play,
  AVG(epa_pass) AS epa_pass,
  AVG(epa_rush) AS epa_rush,
  AVG(success_rate) AS success_rate,
  AVG(pass_success) AS pass_success,
  AVG(rush_success) AS rush_success
FROM by_game
GROUP BY season
ORDER BY season;


### What we’re querying here
Rebuild season-level aggregates to study:
- Pace (plays/game, seconds/play)
- Strategy (pass rate, xPass, PROE)
- Formation/tempo (shotgun, no-huddle)
- Efficiency (EPA/play, success rate, explosive rate)
- Scoring (points/game)

This block filters to true plays (play=1, non-deleted) in regular season.


### Use saved queries from `queries/`
We’ll materialize season tables by executing the saved SQL files so the logic is reusable outside this notebook:
- `queries/league/season_trends.sql`
- `queries/league/season_environment_splits.sql`


In [ ]:
-- Build season_trends from saved query
CREATE OR REPLACE TABLE season_trends AS
SELECT * FROM read_sql_file('queries/league/season_trends.sql');

-- Build season_env_splits from saved query
CREATE OR REPLACE TABLE season_env_splits AS
SELECT * FROM read_sql_file('queries/league/season_environment_splits.sql');


In [ ]:
-- Rebuild season trends with PROE and cleaner play filtering
DROP TABLE IF EXISTS season_trends;

CREATE TABLE season_trends AS
WITH plays AS (
  SELECT
    season::INT AS season,
    game_id,
    play_id,
    order_sequence,
    pass::INT AS is_pass,
    rush::INT AS is_rush,
    shotgun::INT AS is_shotgun,
    no_huddle::INT AS is_no_huddle,
    air_yards,
    epa,
    success::INT AS is_success,
    yards_gained,
    xpass,
    pass_oe,
    COALESCE(roof, game_stadium, stadium, '') AS roof_like,
    surface
  FROM pbp
  WHERE season >= 1999 AND season_type = 'REG' AND COALESCE(play_deleted,0)=0 AND play = 1
), by_game AS (
  SELECT
    season,
    game_id,
    COUNT(*) AS plays,
    SUM(is_pass) AS pass_plays,
    SUM(is_rush) AS rush_plays,
    AVG(is_shotgun) AS shotgun_rate,
    AVG(is_no_huddle) AS no_huddle_rate,
    AVG(air_yards) AS air_yards,
    AVG(epa) AS epa_play,
    AVG(CASE WHEN is_pass=1 THEN epa END) AS epa_pass,
    AVG(CASE WHEN is_rush=1 THEN epa END) AS epa_rush,
    AVG(is_success) AS success_rate,
    AVG(CASE WHEN is_pass=1 THEN is_success END) AS pass_success,
    AVG(CASE WHEN is_rush=1 THEN is_success END) AS rush_success,
    AVG(xpass) AS xpass,
    AVG(pass_oe) AS pass_oe,
    AVG(CASE WHEN yards_gained >= 20 THEN 1 ELSE 0 END) AS explosive_rate,
    AVG(CASE WHEN roof_like ILIKE '%dome%' OR roof_like ILIKE '%closed%' THEN 1 ELSE 0 END) AS is_dome,
    CASE WHEN surface ILIKE '%turf%' OR surface ILIKE '%artificial%' THEN 'turf' ELSE 'grass' END AS surface_class
  FROM plays
  GROUP BY season, game_id, surface_class
), pace AS (
  SELECT
    p.season,
    p.game_id,
    MEDIAN(delta) FILTER (WHERE delta BETWEEN 0 AND 45) AS median_sec_per_play,
    AVG(delta)    FILTER (WHERE delta BETWEEN 0 AND 45) AS mean_sec_per_play
  FROM (
    SELECT
      season,
      game_id,
      order_sequence,
      LAG(game_seconds_remaining) OVER (PARTITION BY season, game_id ORDER BY order_sequence) - game_seconds_remaining AS delta
    FROM (
      SELECT season, game_id, order_sequence, game_seconds_remaining
      FROM pbp
      WHERE season >= 1999 AND season_type='REG' AND COALESCE(play_deleted,0)=0 AND play=1
    ) s
  ) p
  GROUP BY p.season, p.game_id
), by_game_scoring AS (
  SELECT
    season,
    game_id,
    MAX(total_home_score) + MAX(total_away_score) AS total_points
  FROM pbp
  WHERE season >= 1999 AND season_type='REG'
  GROUP BY season, game_id
)
SELECT
  g.season,
  COUNT(DISTINCT g.game_id) AS games,
  AVG(g.plays) AS avg_plays_per_game,
  AVG(g.pass_plays::DOUBLE)/NULLIF(AVG(g.plays::DOUBLE),0) AS pass_rate,
  AVG(g.shotgun_rate) AS shotgun_rate,
  AVG(g.no_huddle_rate) AS no_huddle_rate,
  AVG(g.air_yards) AS air_yards,
  AVG(g.epa_play) AS epa_play,
  AVG(g.epa_pass) AS epa_pass,
  AVG(g.epa_rush) AS epa_rush,
  AVG(g.success_rate) AS success_rate,
  AVG(g.pass_success) AS pass_success,
  AVG(g.rush_success) AS rush_success,
  AVG(g.xpass) AS xpass,
  AVG(g.pass_oe) AS pass_oe,
  AVG(g.explosive_rate) AS explosive_rate,
  AVG(p.median_sec_per_play) AS median_sec_per_play,
  AVG(p.mean_sec_per_play) AS mean_sec_per_play,
  AVG(s.total_points) AS points_per_game
FROM by_game g
LEFT JOIN pace p USING (season, game_id)
LEFT JOIN by_game_scoring s USING (season, game_id)
GROUP BY g.season
ORDER BY g.season;


### What we’re querying here
Compute per-season environment splits to answer:
- Do domes increase scoring and efficiency vs outdoors?
- Does turf elevate EPA/explosive rates vs grass?
- Are these effects stable or changing over time?


In [ ]:
-- Environment splits per season (dome vs outdoor, grass vs turf)
CREATE OR REPLACE TABLE season_env_splits AS
WITH plays AS (
  SELECT
    season::INT AS season,
    game_id,
    pass::INT AS is_pass,
    rush::INT AS is_rush,
    epa,
    success::INT AS is_success,
    yards_gained,
    COALESCE(roof, game_stadium, stadium, '') AS roof_like,
    surface
  FROM pbp
  WHERE season >= 1999 AND season_type = 'REG' AND COALESCE(play_deleted,0)=0 AND play=1
), by_game AS (
  SELECT
    season,
    game_id,
    AVG(CASE WHEN roof_like ILIKE '%dome%' OR roof_like ILIKE '%closed%' THEN 1 ELSE 0 END) AS is_dome,
    CASE WHEN surface ILIKE '%turf%' OR surface ILIKE '%artificial%' THEN 'turf' ELSE 'grass' END AS surface_class,
    COUNT(*) AS plays,
    AVG(epa) AS epa_play,
    AVG(CASE WHEN is_pass=1 THEN epa END) AS epa_pass,
    AVG(CASE WHEN is_rush=1 THEN epa END) AS epa_rush,
    AVG(is_success) AS success_rate,
    AVG(CASE WHEN yards_gained >= 20 THEN 1 ELSE 0 END) AS explosive_rate
  FROM plays
  GROUP BY season, game_id, surface_class
), by_game_scoring AS (
  SELECT
    season,
    game_id,
    MAX(total_home_score) + MAX(total_away_score) AS total_points
  FROM pbp
  WHERE season >= 1999 AND season_type='REG'
  GROUP BY season, game_id
)
SELECT
  season,
  CASE WHEN is_dome >= 0.5 THEN 'dome' ELSE 'outdoor' END AS roof_class,
  surface_class,
  COUNT(*) AS games,
  AVG(plays) AS plays_per_game,
  AVG(epa_play) AS epa_play,
  AVG(epa_pass) AS epa_pass,
  AVG(epa_rush) AS epa_rush,
  AVG(success_rate) AS success_rate,
  AVG(explosive_rate) AS explosive_rate,
  AVG(s.total_points) AS points_per_game
FROM by_game g
LEFT JOIN by_game_scoring s USING (season, game_id)
GROUP BY season, roof_class, surface_class
ORDER BY season, roof_class, surface_class;


### What we’ll visualize next
We’ll chart key macro trends and save shareable outputs:
- Pace: plays/game and median seconds/play (are teams going faster?)
- Strategy: pass rate, xPass, PROE; shotgun and no-huddle usage
- Efficiency: EPA/play (overall/pass/rush) and points/game
- Then compare environment splits (dome vs outdoor, turf vs grass) over the last 10 seasons.


In [ ]:
# Load results and plot/save key trends
st = con.execute("SELECT * FROM season_trends ORDER BY season").pl()
se = con.execute(
    "SELECT * FROM season_env_splits ORDER BY season, roof_class, surface_class"
).pl()

st_pd = st.to_pandas()
se_pd = se.to_pandas()

# Save CSVs for reuse
st_pd.to_csv(REPORT_DIR / "season_trends.csv", index=False)
se_pd.to_csv(REPORT_DIR / "season_env_splits.csv", index=False)

fig, axs = plt.subplots(3, 2, figsize=(14, 12), constrained_layout=True)

# Pace and pass rate
sns.lineplot(ax=axs[0, 0], data=st_pd, x="season", y="avg_plays_per_game", marker="o")
axs[0, 0].set_title("Plays per game")

sns.lineplot(ax=axs[0, 1], data=st_pd, x="season", y="median_sec_per_play", marker="o")
axs[0, 1].invert_yaxis()
axs[0, 1].set_title("Median seconds per play (lower = faster)")

# Strategy
sns.lineplot(
    ax=axs[1, 0], data=st_pd, x="season", y="pass_rate", marker="o", label="Pass rate"
)
sns.lineplot(ax=axs[1, 0], data=st_pd, x="season", y="xpass", marker="o", label="xPass")
sns.lineplot(
    ax=axs[1, 0], data=st_pd, x="season", y="pass_oe", marker="o", label="PROE"
)
axs[1, 0].set_title("Pass tendency (rate/xPass/PROE)")
axs[1, 0].legend()

sns.lineplot(
    ax=axs[1, 1], data=st_pd, x="season", y="shotgun_rate", marker="o", label="Shotgun"
)
sns.lineplot(
    ax=axs[1, 1],
    data=st_pd,
    x="season",
    y="no_huddle_rate",
    marker="o",
    label="No-huddle",
)
axs[1, 1].set_title("Formation/tempo usage")
axs[1, 1].legend()

# Efficiency
sns.lineplot(
    ax=axs[2, 0], data=st_pd, x="season", y="epa_play", marker="o", label="EPA/play"
)
sns.lineplot(
    ax=axs[2, 0], data=st_pd, x="season", y="epa_pass", marker="o", label="EPA/pass"
)
sns.lineplot(
    ax=axs[2, 0], data=st_pd, x="season", y="epa_rush", marker="o", label="EPA/rush"
)
axs[2, 0].set_title("Efficiency over time")
axs[2, 0].legend()

sns.lineplot(ax=axs[2, 1], data=st_pd, x="season", y="points_per_game", marker="o")
axs[2, 1].set_title("Points per game")

plt.suptitle("NFL macro trends (1999–present)", y=1.02)
plt.savefig(REPORT_DIR / "macro_trends_overview.png", dpi=150)
plt.show()

# Environment comparison recent 10 seasons
recent = se_pd[se_pd["season"] >= (se_pd["season"].max() - 9)]
fig, axs = plt.subplots(2, 2, figsize=(12, 8), constrained_layout=True)

sns.lineplot(
    ax=axs[0, 0],
    data=recent[recent["roof_class"] == "dome"],
    x="season",
    y="points_per_game",
    label="Dome",
)
sns.lineplot(
    ax=axs[0, 0],
    data=recent[recent["roof_class"] == "outdoor"],
    x="season",
    y="points_per_game",
    label="Outdoor",
)
axs[0, 0].set_title("Points per game: dome vs outdoor")
axs[0, 0].legend()

sns.lineplot(
    ax=axs[0, 1],
    data=recent[recent["roof_class"] == "dome"],
    x="season",
    y="epa_play",
    label="Dome",
)
sns.lineplot(
    ax=axs[0, 1],
    data=recent[recent["roof_class"] == "outdoor"],
    x="season",
    y="epa_play",
    label="Outdoor",
)
axs[0, 1].set_title("EPA/play: dome vs outdoor")
axs[0, 1].legend()

sns.lineplot(
    ax=axs[1, 0],
    data=recent[recent["surface_class"] == "turf"],
    x="season",
    y="points_per_game",
    label="Turf",
)
sns.lineplot(
    ax=axs[1, 0],
    data=recent[recent["surface_class"] == "grass"],
    x="season",
    y="points_per_game",
    label="Grass",
)
axs[1, 0].set_title("Points per game: turf vs grass")
axs[1, 0].legend()

sns.lineplot(
    ax=axs[1, 1],
    data=recent[recent["surface_class"] == "turf"],
    x="season",
    y="epa_play",
    label="Turf",
)
sns.lineplot(
    ax=axs[1, 1],
    data=recent[recent["surface_class"] == "grass"],
    x="season",
    y="epa_play",
    label="Grass",
)
axs[1, 1].set_title("EPA/play: turf vs grass")
axs[1, 1].legend()

plt.savefig(REPORT_DIR / "environment_comparisons.png", dpi=150)
plt.show()

st.tail(5)

## Suggested interpretations for Best Ball strategy

- Faster pace (lower seconds/play and higher plays/game) increases weekly spike potential. Stack teams that rank top-10 in pace and no-huddle usage.
- Higher PROE and shotgun rates correlate with higher passing volume variance. Lean into WR-heavy builds when macro pass tendency is rising.
- Dome advantage: if dome splits consistently show higher points/EPA, prioritize dome WR/QB for playoff weeks and draft tiebreakers.
- Turf vs grass: if turf shows higher points/epa/explosive rate, mildly boost deep-threat WRs and K/QB in those environments.
- Air yards trend: rising air yards supports spike-week WR archetypes; falling trend supports RB/TE floor builds.
- Keep playoff weeks (Weeks 15-17) in mind. Prefer players on teams with dome home games or favorable late-season surfaces.
